# Answers changed
## Load dependencies and data

In [55]:
import pandas as pd
import json

Needs to be updated to new/folder data structure, load the para file basically

# Prepare paradata

In [59]:
df_para['answer_changed'] = False
group_columns = ['interview__id', 'param', 'roster_level']  # group dfs by interview, variable and roster instance

## List questions

In [60]:
df_para_list = df_para[(df_para['type'] == 'TextListQuestion') & (df_para['event'] == 'AnswerSet')].copy()
grouped_list = df_para_list.groupby(group_columns)
for _, group in grouped_list:
    prev_answers = set()  # set an empty set for previous answers
    for index, row in group.iterrows():
        row_answers = set(row['answer'].split('|')) if pd.notnull(row['answer']) else set()
        if prev_answers.difference(row_answers):
            df_para_list.at[
                index, 'answer_changed'] = True  # can be removed, just to verify more easily
            df_para.at[index, 'answer_changed'] = True
        prev_answers = row_answers

Currently counting any change, irrespective if it is deleting a list item, or changing the spelling or updating the list item. There is a lot of the latter, which is less of an issue and more a sign of diligence.

I think we would like to redefine how change answer and remove answer are defined. E.g consider as "removed answer" if unselecting a multi_select, changing yes/no, or removing a list item? These actions normally mean they selected something and then decided to unselect or list it (as always, could be down to the respondent). These actions are worse if they trigger a lot of question enablement (which we do not know for now).

## Single answer questions

In [61]:
df_para_question = df_para[(~df_para['type'].isin(['YesNoQuestion', 'MultyOptionsQuestion', 'TextListQuestion', 'Variable'])) & (df_para['event'] == 'AnswerSet')].copy()
df_para_question = df_para_question[df_para_question.duplicated(subset=group_columns, keep=False)]
if df_para_question.shape[0] > 0:
    grouped_question = df_para_question.groupby(group_columns)
    for _, group in grouped_question:
        prev_answer = None  # set an empty answer for previous answers
        for index, row in group.iterrows():
            row_answer = row['answer']
            if prev_answer is not None and prev_answer != row_answer:
                df_para_question.at[index, 'answer_changed'] = True  # can be removed, just to verify more easily
                df_para.at[index, 'answer_changed'] = True
            prev_answer = row_answer

@Gabriele, above cell takes quite long to execute, can we write this faster?

## YesNo questions

In [62]:
df_para_yesno = df_para[(df_para['type'] == 'YesNoQuestion') & (df_para['event'] == 'AnswerSet')].copy()
if df_para_yesno.shape[0] > 0:
    df_para_yesno[['yes_answers', 'no_answers']] = df_para_yesno['answer'].str.split('|', expand=True)
    grouped_yesno = df_para_yesno.groupby(group_columns)

    for _, group in grouped_yesno:
        prev_yes_answers = set()  # set an empty set for previous yes-answers
        for index, row in group.iterrows():
            yes_answers = set(row['yes_answers'].split(', ')) if pd.notnull(row['yes_answers']) else set()
            no_answers = set(row['no_answers'].split(', ')) if pd.notnull(row['no_answers']) else set()

            if len(prev_yes_answers.intersection(no_answers)) > 0:
                df_para_yesno.at[
                    index, 'answer_changed'] = True  # can be removed, just to verify more easily
                df_para.at[index, 'answer_changed'] = True
            prev_yes_answers = yes_answers

## Normal multi-answer questions

In [63]:
df_para_multi = df_para[(df_para['type'] == 'MultyOptionsQuestion') & (df_para['event'] == 'AnswerSet')].copy()
if df_para_multi.shape[0] > 0:
    grouped_multi = df_para_multi.groupby(group_columns)
    for _, group in grouped_multi:
        prev_answers = set()  # set an empty set for previous answers
        for index, row in group.iterrows():
            row_answers = set(row['answer'].split(', ')) if pd.notnull(row['answer']) else set()
            if prev_answers.difference(row_answers):
                df_para_multi.at[
                    index, 'answer_changed'] = True  # can be removed, just to verify more easily
                df_para.at[index, 'answer_changed'] = True
            prev_answers = row_answers

# Summary

In [ ]:
summarize_group = ['interview__id', 'param']
answers_changed = df_para[df_para['answer_changed']].groupby(summarize_group).size().reset_index(name='answers_changed')
#interview_questions = pd.DataFrame(df_para[df_para['event'] == 'AnswerSet'][summarize_group].drop_duplicates())
#n_answer_changed = interview_questions.merge(n_answer_changed, on=summarize_group, how='left').fillna({'n_answer_changed': 0})
# @Gabriele, so we need the 0 counts, if so, outcomment above?
answers_changed